# Probability with SciPy - Professional Statistical Computing

## Introduction

Now that we've implemented probability functions from scratch, let's learn **professional tools** used in industry and research.

**SciPy** is the industry-standard library for scientific computing in Python. Its `scipy.stats` module provides:
- Probability distributions (discrete and continuous)
- Statistical functions
- Hypothesis tests
- Random sampling

### Why SciPy?
- **Fast**: Optimized C/Fortran code
- **Reliable**: Extensively tested
- **Complete**: Hundreds of distributions
- **Standard**: Used across academia and industry

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
from scipy.special import comb, perm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Libraries imported!")
print(f"SciPy version: {stats.__version__ if hasattr(stats, '__version__') else 'Available'}")

## 1. Discrete Probability Distributions

### Bernoulli Distribution
Binary outcomes: success/failure (e.g., seed germinates or not)

In [ ]:
# Bernoulli: Single seed germination
p_germination = 0.85
bernoulli_dist = stats.bernoulli(p=p_germination)

print("Bernoulli Distribution: Single Seed Germination")
print("="*60)
print(f"P(Success) = {p_germination}")
print(f"P(X=1) [germinate] = {bernoulli_dist.pmf(1):.2f}")
print(f"P(X=0) [fail] = {bernoulli_dist.pmf(0):.2f}")
print(f"Mean = {bernoulli_dist.mean():.2f}")
print(f"Variance = {bernoulli_dist.var():.4f}")

# Sample outcomes
samples = bernoulli_dist.rvs(size=10)
print(f"\n10 random trials: {samples}")
print(f"Success rate: {samples.mean():.1%}")

### Binomial Distribution
Number of successes in n independent trials

In [ ]:
# Binomial: Multiple seeds
n_seeds = 20
p_germ = 0.85
binomial_dist = stats.binom(n=n_seeds, p=p_germ)

print(f"Binomial Distribution: {n_seeds} Seeds, {p_germ:.0%} Success Rate")
print("="*60)
print(f"Expected germinations: {binomial_dist.mean():.1f}")
print(f"Standard deviation: {binomial_dist.std():.2f}")
print(f"\nP(exactly 15 germinate) = {binomial_dist.pmf(15):.4f}")
print(f"P(at least 18 germinate) = {binomial_dist.sf(17):.4f}")  # sf = survival function = 1 - CDF
print(f"P(at most 12 germinate) = {binomial_dist.cdf(12):.4f}")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# PMF
x = np.arange(0, n_seeds + 1)
pmf = binomial_dist.pmf(x)
ax1.bar(x, pmf, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(binomial_dist.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean = {binomial_dist.mean():.1f}')
ax1.set_xlabel('Number of Germinations')
ax1.set_ylabel('Probability')
ax1.set_title('Binomial PMF: Probability Mass Function')
ax1.legend()

# CDF
cdf = binomial_dist.cdf(x)
ax2.step(x, cdf, where='post', color='green', linewidth=2)
ax2.set_xlabel('Number of Germinations')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Binomial CDF: Cumulative Distribution')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Poisson Distribution
Count of rare events in fixed interval (e.g., pest outbreaks per season)

In [ ]:
# Poisson: Pest outbreaks
lambda_rate = 2.5  # Average outbreaks per season
poisson_dist = stats.poisson(mu=lambda_rate)

print(f"Poisson Distribution: Pest Outbreaks (λ={lambda_rate})")
print("="*60)
print(f"Mean = {poisson_dist.mean():.2f}")
print(f"Variance = {poisson_dist.var():.2f}")
print(f"\nP(exactly 2 outbreaks) = {poisson_dist.pmf(2):.4f}")
print(f"P(3 or more outbreaks) = {poisson_dist.sf(2):.4f}")
print(f"P(0 outbreaks) = {poisson_dist.pmf(0):.4f}")

# Visualize
x = np.arange(0, 10)
pmf = poisson_dist.pmf(x)
plt.figure(figsize=(10, 5))
plt.bar(x, pmf, color='salmon', edgecolor='black', alpha=0.7)
plt.axvline(lambda_rate, color='red', linestyle='--', linewidth=2, label=f'Mean = {lambda_rate}')
plt.xlabel('Number of Outbreaks')
plt.ylabel('Probability')
plt.title('Poisson Distribution: Rare Events')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Continuous Probability Distributions

### Uniform Distribution
Equal probability across a range

In [ ]:
# Uniform: Random sampling time
a, b = 0, 24  # Hours in a day
uniform_dist = stats.uniform(loc=a, scale=b-a)

print("Uniform Distribution: Random Sampling Time (0-24 hours)")
print("="*60)
print(f"Mean = {uniform_dist.mean():.1f} hours")
print(f"P(sample before noon) = {uniform_dist.cdf(12):.2f}")
print(f"P(sample between 8am-5pm) = {uniform_dist.cdf(17) - uniform_dist.cdf(8):.2f}")

# Visualize
x = np.linspace(a, b, 1000)
pdf = uniform_dist.pdf(x)
plt.figure(figsize=(10, 5))
plt.plot(x, pdf, 'b-', linewidth=2, label='PDF')
plt.fill_between(x, 0, pdf, alpha=0.3)
plt.xlabel('Time (hours)')
plt.ylabel('Probability Density')
plt.title('Uniform Distribution: Equal Probability')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Normal (Gaussian) Distribution
The most important distribution in statistics

In [ ]:
# Normal: Crop yields
mean_yield = 75  # bushels/acre
std_yield = 10
normal_dist = stats.norm(loc=mean_yield, scale=std_yield)

print(f"Normal Distribution: Crop Yield (μ={mean_yield}, σ={std_yield})")
print("="*60)
print(f"P(yield > 85) = {normal_dist.sf(85):.4f}")
print(f"P(60 < yield < 90) = {normal_dist.cdf(90) - normal_dist.cdf(60):.4f}")
print(f"P(yield < 50) = {normal_dist.cdf(50):.4f}")
print(f"\n68% confidence interval: [{normal_dist.ppf(0.16):.1f}, {normal_dist.ppf(0.84):.1f}]")
print(f"95% confidence interval: [{normal_dist.ppf(0.025):.1f}, {normal_dist.ppf(0.975):.1f}]")

# Visualize
x = np.linspace(mean_yield - 4*std_yield, mean_yield + 4*std_yield, 1000)
pdf = normal_dist.pdf(x)

plt.figure(figsize=(12, 6))
plt.plot(x, pdf, 'b-', linewidth=2, label='PDF')
plt.fill_between(x, 0, pdf, alpha=0.3)

# Mark special regions
x_68 = np.linspace(mean_yield - std_yield, mean_yield + std_yield, 100)
plt.fill_between(x_68, 0, normal_dist.pdf(x_68), color='green', alpha=0.3, label='68% (±1σ)')

x_95 = np.linspace(mean_yield - 2*std_yield, mean_yield + 2*std_yield, 100)
plt.fill_between(x_95, 0, normal_dist.pdf(x_95), color='yellow', alpha=0.2, label='95% (±2σ)')

plt.axvline(mean_yield, color='red', linestyle='--', label=f'Mean = {mean_yield}')
plt.xlabel('Yield (bushels/acre)')
plt.ylabel('Probability Density')
plt.title('Normal Distribution: The Bell Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Monte Carlo Simulation

Use random sampling to estimate probabilities for complex scenarios

In [ ]:
# Monte Carlo: Complex farm profit simulation
print("Monte Carlo Simulation: Farm Profit Estimation")
print("="*60)

n_simulations = 10000

# Random variables
yields = stats.norm(loc=75, scale=10).rvs(size=n_simulations)
prices = stats.norm(loc=5.5, scale=0.8).rvs(size=n_simulations)
costs = stats.norm(loc=250, scale=30).rvs(size=n_simulations)

# Calculate profit
acres = 100
profits = acres * (yields * prices - costs)

# Analyze results
print(f"\nSimulated {n_simulations:,} farm seasons:")
print(f"Mean profit: ${profits.mean():,.0f}")
print(f"Std deviation: ${profits.std():,.0f}")
print(f"\nP(profit > $10,000) = {(profits > 10000).mean():.1%}")
print(f"P(loss) = {(profits < 0).mean():.1%}")
print(f"\n95% confidence interval: [${np.percentile(profits, 2.5):,.0f}, ${np.percentile(profits, 97.5):,.0f}]")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(profits, bins=50, color='skyblue', edgecolor='black', alpha=0.7, density=True)
ax1.axvline(profits.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean = ${profits.mean():,.0f}')
ax1.axvline(0, color='orange', linestyle='--', linewidth=2, label='Break-even')
ax1.set_xlabel('Profit ($)')
ax1.set_ylabel('Probability Density')
ax1.set_title('Monte Carlo: Profit Distribution')
ax1.legend()

# Cumulative probability
sorted_profits = np.sort(profits)
cumulative = np.arange(1, len(sorted_profits) + 1) / len(sorted_profits)
ax2.plot(sorted_profits, cumulative, 'g-', linewidth=2)
ax2.axvline(0, color='orange', linestyle='--', linewidth=2, label='Break-even')
ax2.set_xlabel('Profit ($)')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Cumulative Distribution')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

## 4. Statistical Tests for Independence

Use chi-square test to determine if two categorical variables are independent

In [ ]:
# Chi-square test: Irrigation vs. Yield
# Contingency table: rows = irrigation, columns = yield level
observed = np.array([
    [45, 35, 20],  # Irrigated: [High, Medium, Low yield]
    [20, 40, 40]   # Not irrigated: [High, Medium, Low yield]
])

print("Chi-Square Test: Irrigation vs. Yield Level")
print("="*60)
print("\nObserved Counts:")
print(pd.DataFrame(observed, 
                   index=['Irrigated', 'Not Irrigated'],
                   columns=['High Yield', 'Medium Yield', 'Low Yield']))

# Perform chi-square test
chi2, p_value, dof, expected = stats.chi2_contingency(observed)

print(f"\nTest Results:")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")

if p_value < 0.05:
    print("\n✓ DEPENDENT: Irrigation and yield are significantly related (p < 0.05)")
else:
    print("\n✗ INDEPENDENT: No significant relationship found (p ≥ 0.05)")

print("\nExpected counts (if independent):")
print(pd.DataFrame(expected,
                   index=['Irrigated', 'Not Irrigated'],
                   columns=['High Yield', 'Medium Yield', 'Low Yield']).round(1))

## 5. Comparing From-Scratch vs. SciPy

Let's compare our implementations with SciPy

In [ ]:
# Import our module
import sys
sys.path.append('../2_from_scratch')
import probability_functions as pf

print("Comparison: From-Scratch vs. SciPy")
print("="*60)

# Test 1: Binomial probability
n, p, k = 20, 0.85, 15
scipy_result = stats.binom(n, p).pmf(k)
manual = comb(n, k) * (p**k) * ((1-p)**(n-k))
print(f"\n1. P(X=15) for Binomial(n=20, p=0.85):")
print(f"   SciPy: {scipy_result:.6f}")
print(f"   Manual: {manual:.6f}")
print(f"   Match: {np.isclose(scipy_result, manual)} ✓")

# Test 2: Conditional probability
P_AB, P_B = 0.12, 0.30
our_result = pf.conditional_probability(P_AB, P_B)
scipy_result = P_AB / P_B
print(f"\n2. Conditional Probability P(A|B):")
print(f"   Our function: {our_result:.4f}")
print(f"   Direct calculation: {scipy_result:.4f}")
print(f"   Match: {np.isclose(our_result, scipy_result)} ✓")

# Performance comparison
import time
print(f"\n3. Performance (10,000 iterations):")

start = time.time()
for _ in range(10000):
    pf.calculate_probability(85, 100)
our_time = time.time() - start

start = time.time()
for _ in range(10000):
    85 / 100
builtin_time = time.time() - start

print(f"   Our function: {our_time:.4f}s")
print(f"   Built-in: {builtin_time:.4f}s")
print(f"   ⚡ SciPy optimizations make a difference at scale!")

print("\n💡 Key Takeaway: Our implementations are correct, but SciPy is:")
print("   • Faster (optimized C code)")
print("   • More comprehensive (hundreds of distributions)")
print("   • Better tested (decades of use)")
print("   • Industry standard (use in production)")

## Summary

### What We Learned

**Discrete Distributions:**
- Bernoulli: Single trial
- Binomial: Multiple trials
- Poisson: Rare events

**Continuous Distributions:**
- Uniform: Equal probability
- Normal: Bell curve (most important!)

**SciPy Functions:**
- `.pmf()` / `.pdf()`: Probability mass/density
- `.cdf()`: Cumulative distribution
- `.sf()`: Survival function (1 - CDF)
- `.ppf()`: Percent point function (inverse CDF)
- `.rvs()`: Random samples
- `.mean()`, `.var()`, `.std()`: Statistics

**Applications:**
- Monte Carlo simulation for complex scenarios
- Chi-square test for independence
- Confidence intervals
- Probability calculations

### Next Steps

Continue to: `statistical_functions.ipynb` for advanced SciPy features, then apply everything to real agricultural problems!